In [1]:
from anomaly import io, tmm, adm
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

import anomaly.utils.modelselect_utils as mu
import anomaly.utils.statsutils as su

In [2]:
predictor_dict = {
"naive_predictor" : tmm.NaivePredictor(),
"ar_predictor" : tmm.AR(),
"ma_predictor" : tmm.MA(),
"arma_predictor" : tmm.ARMA(),
"poly_predictor" : tmm.Polynomial(),
}

### Compute the features and the scores of the models for each time series in the benchmark

In [ ]:
%%capture --no-stdout

for benchmark_index in range(2,3):
    bench = io.BenchmarkDataset(benchmark_index)

    ## for debug
    #bench.len = 5

    score_dict = mu.compute_predictor_scores(predictor_dict, bench, detector=adm.KSigma())
    score_df = pd.concat([score_dict[model_name].assign(model_name=model_name)
                      for model_name in score_dict.keys()])
    
    score_df.to_csv(f"saved_data/score_df_{benchmark_index}.csv", index_label="ts_index")
    
    features = mu.compute_benchmark_features(bench)
    features.to_csv(f"saved_data/features_{benchmark_index}.csv", index_label="ts_index")

 96%|██████████████████████████████████████████████████▉  | 96/100 [05:11<00:14,  3.70s/it]

### Analyse the results

In [6]:
score_df = pd.read_csv(f"saved_data/score_df_{benchmark_index}.csv", index_col="ts_index")
features = pd.read_csv(f"saved_data/features_{benchmark_index}.csv", index_col="ts_index")

In [7]:
features.head()

,periodicity,trend_score,seasonality_score,nonlinearity,skew,kurtosis,lyapunov
ts_index,,,,,,,
0,1.0,1.000000,0.000000,0.934517,0.083447,-0.719147,-0.000418
1,77.0,-0.077289,-3.662810,1.014550,0.045395,-0.432972,-0.000689
2,157.0,-1.737175,-0.312640,1.014645,0.011550,-0.610619,0.000884
3,35.0,-0.021986,-21.816881,0.368837,0.124895,-0.883236,-0.001314
4,199.0,-0.029197,-3.816528,0.714450,0.101515,-0.827419,-0.000919


In [8]:
score_df = pd.read_csv(f"saved_data/score_df_{benchmark_index}.csv")

In [9]:
score_df

,ts_index,bias,mad,mape,mse,sae,f1,recall,model_name
0,0,-3.793342e+00,219.740122,0.364106,9.438015e+04,3.122507e+05,0.400000,1.000000,naive_predictor
1,1,-2.891046e+00,672.837596,1.509236,7.226477e+05,9.561022e+05,0.285714,1.000000,naive_predictor
2,2,-1.933118e+00,323.333956,0.955834,1.808965e+05,4.594576e+05,0.285714,1.000000,naive_predictor
3,3,-1.002302e+01,104.670272,0.034638,5.605039e+05,1.487365e+05,0.400000,0.333333,naive_predictor
4,4,-5.177551e+00,391.676889,0.404856,3.822769e+05,5.565729e+05,0.400000,0.333333,naive_predictor
5,0,-6.063748e+00,193.402067,0.307673,7.465488e+04,2.748243e+05,0.333333,1.000000,ar_predictor
6,1,-6.552432e+00,518.378271,1.126090,4.397421e+05,7.366155e+05,0.400000,1.000000,ar_predictor
7,2,-3.808951e+00,264.172482,0.722112,1.218326e+05,3.753891e+05,0.222222,1.000000,ar_predictor
8,3,-9.200717e+00,135.081588,0.184265,4.620099e+05,1.919509e+05,0.315789,0.333333,ar_predictor
9,4,-7.522152e+00,363.278187,0.352766,3.291403e+05,5.162183e+05,0.545455,0.666667,ar_predictor


In [10]:
score_df.groupby("model_name").mean()

,ts_index,bias,mad,mape,mse,sae,f1,recall
model_name,,,,,,,,
ar_predictor,2.0,-6.629600e+00,294.862519,0.538581,2.854759e+05,4.189996e+05,0.363360,0.800000
arma_predictor,2.0,-3.478542e+00,290.879887,0.514362,2.503081e+05,4.133403e+05,0.490303,0.866667
ma_predictor,2.0,7.619852e-01,507.910396,0.985939,5.207891e+05,7.217407e+05,0.432982,0.800000
naive_predictor,2.0,-4.763615e+00,342.451767,0.653734,3.881410e+05,4.866240e+05,0.354286,0.733333
poly_predictor,2.0,2.362798e-12,918.887485,1.658767,1.268022e+06,1.305739e+06,1.000000,1.000000
